In [ ]:
import os
import numpy as np
import pandas as pd
from statsmodels.tsa.seasonal import STL
from tqdm import tqdm
import multiprocessing
from joblib import Parallel, delayed

In [2]:
import matplotlib.pyplot as plt
plt.rcParams.update(**{'figure.dpi': 150})

# Sales and Categories

In [3]:
sales_train_validation = pd.read_csv('./datasets/m5-forecasting-accuracy/sales_train_validation.csv')
#sales_train_validation = sales_train_validation[sales_train_validation['dept_id']!='FOODS_3']
#sales_train_validation = sales_train_validation[sales_train_validation['dept_id']!='HOUSEHOLD_2']
display(sales_train_validation)

,id,item_id,dept_id,cat_id,store_id,state_id,d_1,d_2,d_3,d_4,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,0,0,2,2,...,2,0,0,0,0,0,1,0,0,1
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,0,6,0,2,...,2,1,0,2,0,1,0,0,1,0
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


# Sell prices

In [4]:
sell_prices = np.load('./datasets/m5-forecasting-accuracy/sell_prices_individual.npy', allow_pickle=True)

def merge_sell_prices(i):
    info = list(sales_train_validation.iloc[i].values[:6])
    prices = list(sell_prices[i])
    res = info + prices
    del info
    del prices
    return res

In [5]:
rows = []
for i in tqdm(range(len(sales_train_validation)), total=len(sales_train_validation)):
    rows.append(merge_sell_prices(i))

100%|██████████| 30490/30490 [00:13<00:00, 2244.27it/s]


In [6]:
columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
columns = columns + [f's_{i}' for i in range(1913)]
sell_prices = pd.DataFrame(rows, columns=columns)
display(sell_prices)

,id,item_id,dept_id,cat_id,store_id,state_id,s_0,s_1,s_2,s_3,...,s_1903,s_1904,s_1905,s_1906,s_1907,s_1908,s_1909,s_1910,s_1911,s_1912
0,HOBBIES_1_001_CA_1_validation,HOBBIES_1_001,HOBBIES_1,HOBBIES,CA_1,CA,NaN,NaN,NaN,NaN,...,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38,8.38
1,HOBBIES_1_002_CA_1_validation,HOBBIES_1_002,HOBBIES_1,HOBBIES,CA_1,CA,NaN,NaN,NaN,NaN,...,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97,3.97
2,HOBBIES_1_003_CA_1_validation,HOBBIES_1_003,HOBBIES_1,HOBBIES,CA_1,CA,NaN,NaN,NaN,NaN,...,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97,2.97
3,HOBBIES_1_004_CA_1_validation,HOBBIES_1_004,HOBBIES_1,HOBBIES,CA_1,CA,NaN,NaN,NaN,NaN,...,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64,4.64
4,HOBBIES_1_005_CA_1_validation,HOBBIES_1_005,HOBBIES_1,HOBBIES,CA_1,CA,NaN,NaN,NaN,NaN,...,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88,2.88
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_validation,FOODS_3_823,FOODS_3,FOODS,WI_3,WI,2.50,2.50,2.50,2.50,...,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98,2.98
30486,FOODS_3_824_WI_3_validation,FOODS_3_824,FOODS_3,FOODS,WI_3,WI,2.47,2.47,2.47,2.47,...,2.48,2.48,2.48,2.48,2.48,2.48,2.48,2.48,2.48,2.48
30487,FOODS_3_825_WI_3_validation,FOODS_3_825,FOODS_3,FOODS,WI_3,WI,4.00,4.00,4.00,4.00,...,3.98,3.98,3.98,3.98,3.98,3.98,3.98,3.98,3.98,3.98
30488,FOODS_3_826_WI_3_validation,FOODS_3_826,FOODS_3,FOODS,WI_3,WI,NaN,NaN,NaN,NaN,...,1.28,1.28,1.28,1.28,1.28,1.28,1.28,1.28,1.28,1.28


# Aggregation

## Sales

In [7]:
sales_items = sales_train_validation.groupby(['item_id', 'cat_id', 'store_id']).aggregate('sum')
sales_items.reset_index(inplace=True)
print(sales_items['cat_id'].value_counts())
print(sales_items['store_id'].value_counts())
display(sales_items)

FOODS        14370
HOUSEHOLD    10470
HOBBIES       5650
Name: cat_id, dtype: int64
CA_1    3049
CA_2    3049
CA_3    3049
CA_4    3049
TX_1    3049
TX_2    3049
TX_3    3049
WI_1    3049
WI_2    3049
WI_3    3049
Name: store_id, dtype: int64


/tmp/ipykernel_2805728/1487184975.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sales_items = sales_train_validation.groupby(['item_id', 'cat_id', 'store_id']).aggregate('sum')


,item_id,cat_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,FOODS_1_001,FOODS,CA_1,3,0,0,1,4,2,0,...,0,2,0,4,1,1,0,1,1,0
1,FOODS_1_001,FOODS,CA_2,2,0,0,0,1,0,8,...,1,0,14,0,1,1,4,0,0,4
2,FOODS_1_001,FOODS,CA_3,1,2,1,1,1,2,0,...,0,0,13,0,0,0,0,0,1,0
3,FOODS_1_001,FOODS,CA_4,0,1,1,1,1,1,0,...,0,0,0,2,0,0,0,1,1,1
4,FOODS_1_001,FOODS,TX_1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516,HOUSEHOLD,TX_2,0,1,0,0,0,0,1,...,0,1,1,0,0,1,0,0,0,0
30486,HOUSEHOLD_2_516,HOUSEHOLD,TX_3,2,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30487,HOUSEHOLD_2_516,HOUSEHOLD,WI_1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
30488,HOUSEHOLD_2_516,HOUSEHOLD,WI_2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


## Sell prices

In [8]:
sell_prices = sell_prices.groupby(['item_id', 'cat_id', 'store_id']).aggregate('mean').fillna(0)
sell_prices.reset_index(inplace=True)
print(sell_prices['cat_id'].value_counts())
print(sales_items['store_id'].value_counts())
display(sell_prices)

/tmp/ipykernel_2805728/1944782260.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  sell_prices = sell_prices.groupby(['item_id', 'cat_id', 'store_id']).aggregate('mean').fillna(0)


FOODS        14370
HOUSEHOLD    10470
HOBBIES       5650
Name: cat_id, dtype: int64
CA_1    3049
CA_2    3049
CA_3    3049
CA_4    3049
TX_1    3049
TX_2    3049
TX_3    3049
WI_1    3049
WI_2    3049
WI_3    3049
Name: store_id, dtype: int64


,item_id,cat_id,store_id,s_0,s_1,s_2,s_3,s_4,s_5,s_6,...,s_1903,s_1904,s_1905,s_1906,s_1907,s_1908,s_1909,s_1910,s_1911,s_1912
0,FOODS_1_001,FOODS,CA_1,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
1,FOODS_1_001,FOODS,CA_2,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
2,FOODS_1_001,FOODS,CA_3,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
3,FOODS_1_001,FOODS,CA_4,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
4,FOODS_1_001,FOODS,TX_1,2.00,2.00,2.00,2.00,2.00,2.00,2.00,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516,HOUSEHOLD,TX_2,5.94,5.94,5.94,5.94,5.94,5.94,5.94,...,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94
30486,HOUSEHOLD_2_516,HOUSEHOLD,TX_3,5.94,5.94,5.94,5.94,5.94,5.94,5.94,...,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94
30487,HOUSEHOLD_2_516,HOUSEHOLD,WI_1,5.94,5.94,5.94,5.94,5.94,5.94,5.94,...,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94
30488,HOUSEHOLD_2_516,HOUSEHOLD,WI_2,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94


## Merge

In [9]:
sales_items = sales_items.merge(sell_prices, how='left', on=['item_id', 'cat_id', 'store_id'])
display(sales_items)

,item_id,cat_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,...,s_1903,s_1904,s_1905,s_1906,s_1907,s_1908,s_1909,s_1910,s_1911,s_1912
0,FOODS_1_001,FOODS,CA_1,3,0,0,1,4,2,0,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
1,FOODS_1_001,FOODS,CA_2,2,0,0,0,1,0,8,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
2,FOODS_1_001,FOODS,CA_3,1,2,1,1,1,2,0,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
3,FOODS_1_001,FOODS,CA_4,0,1,1,1,1,1,0,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
4,FOODS_1_001,FOODS,TX_1,0,1,1,0,0,0,0,...,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24,2.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516,HOUSEHOLD,TX_2,0,1,0,0,0,0,1,...,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94
30486,HOUSEHOLD_2_516,HOUSEHOLD,TX_3,2,0,0,0,0,0,0,...,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94
30487,HOUSEHOLD_2_516,HOUSEHOLD,WI_1,0,0,0,1,0,0,0,...,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94
30488,HOUSEHOLD_2_516,HOUSEHOLD,WI_2,0,0,0,0,0,0,0,...,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94,5.94


# Drop sales with many zeros

In [10]:
tmp = sales_items.values[:,2:2+1913]
zero_counts = np.array([arr[np.where(arr==0)].size for arr in tmp])
flags = []
for i in range(len(zero_counts)):
    if zero_counts[i] < 400:
        flags.append(i)
print(flags)
del tmp

[104, 105, 106, 160, 161, 162, 163, 165, 167, 168, 169, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 591, 617, 830, 831, 832, 833, 834, 835, 836, 837, 838, 839, 841, 842, 855, 856, 945, 947, 948, 1031, 1052, 1580, 1581, 1582, 1583, 1587, 1588, 1589, 1665, 1666, 1680, 1682, 1691, 1791, 1792, 2140, 2141, 2142, 2143, 2144, 2145, 2146, 2147, 2148, 2149, 2280, 2281, 2282, 2283, 2284, 2285, 2286, 2312, 2319, 2340, 2342, 2343, 2344, 2346, 2450, 2452, 2675, 2692, 2695, 2699, 2746, 2974, 2975, 2976, 3425, 3661, 3662, 3664, 3665, 3666, 3667, 3668, 3782, 3784, 3785, 3786, 3788, 3789, 3858, 3950, 3951, 3952, 3953, 3954, 3955, 3956, 3975, 4110, 4111, 4112, 4113, 4114, 4115, 4116, 4117, 4118, 4119, 4134, 4184, 4185, 4186, 4470, 4472, 4474, 4475, 4476, 4580, 4582, 4583, 4900, 4902, 4903, 4904, 4905, 4906, 4907, 4908, 4909, 4991, 4994, 4996, 4998, 5070, 5072, 5073, 5075, 5079, 5130, 5132, 5133, 5182, 5364, 5365, 5366, 5369, 5390, 5392, 5399, 5404, 5405, 5406, 5409, 5478, 5562, 5565, 5566, 5610, 5

In [11]:
sales_items = sales_items.iloc[flags]
print(sales_items['cat_id'].value_counts())
print(sales_items['store_id'].value_counts())
display(sales_items)

FOODS        858
HOUSEHOLD    256
HOBBIES       36
Name: cat_id, dtype: int64
CA_3    244
CA_1    144
TX_2    130
TX_3    113
TX_1    107
WI_3     94
CA_2     92
WI_2     84
WI_1     74
CA_4     68
Name: store_id, dtype: int64


,item_id,cat_id,store_id,d_1,d_2,d_3,d_4,d_5,d_6,d_7,...,s_1903,s_1904,s_1905,s_1906,s_1907,s_1908,s_1909,s_1910,s_1911,s_1912
104,FOODS_1_012,FOODS,TX_1,0,0,0,0,0,0,0,...,6.27,6.27,6.27,6.27,6.27,6.27,6.27,6.27,6.27,6.27
105,FOODS_1_012,FOODS,TX_2,0,0,0,0,0,0,0,...,6.27,6.27,6.27,6.27,6.27,6.27,6.27,6.27,6.27,6.27
106,FOODS_1_012,FOODS,TX_3,0,0,0,0,0,0,0,...,6.27,6.27,6.27,6.27,6.27,6.27,6.27,6.27,6.27,6.27
160,FOODS_1_018,FOODS,CA_1,17,3,8,2,5,0,1,...,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98
161,FOODS_1_018,FOODS,CA_2,4,1,5,4,3,3,4,...,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26932,HOUSEHOLD_2_160,HOUSEHOLD,CA_3,0,0,0,0,0,0,0,...,3.93,3.93,3.93,3.93,3.93,3.93,3.93,3.93,3.93,3.93
28822,HOUSEHOLD_2_350,HOUSEHOLD,CA_3,5,5,1,5,7,0,7,...,2.94,2.94,2.94,2.94,2.94,2.94,2.94,2.94,2.94,2.94
29412,HOUSEHOLD_2_409,HOUSEHOLD,CA_3,0,0,0,0,0,0,0,...,3.93,3.93,3.93,3.93,3.93,3.93,3.93,3.93,3.93,3.93
29592,HOUSEHOLD_2_427,HOUSEHOLD,CA_3,2,4,0,2,0,2,2,...,3.48,3.48,3.48,3.48,3.48,3.48,3.48,3.48,3.48,3.48


In [12]:
sales_items['cat_store_id'] = sales_items['cat_id'] + '_' + sales_items['store_id']
columns = list(sales_items.columns.values[:3]) + ['cat_store_id'] + list(sales_items.columns.values[3:-1])
sales_items = sales_items[columns]
print(sales_items['cat_store_id'].value_counts())
display(sales_items)

FOODS_CA_3        172
FOODS_CA_1        112
FOODS_TX_2         98
FOODS_TX_3         83
FOODS_TX_1         78
FOODS_WI_3         73
FOODS_CA_2         66
FOODS_CA_4         64
HOUSEHOLD_CA_3     60
FOODS_WI_2         59
FOODS_WI_1         53
HOUSEHOLD_TX_2     30
HOUSEHOLD_TX_1     28
HOUSEHOLD_TX_3     28
HOUSEHOLD_WI_2     25
HOUSEHOLD_CA_2     24
HOUSEHOLD_CA_1     23
HOUSEHOLD_WI_3     19
HOUSEHOLD_WI_1     18
HOBBIES_CA_3       12
HOBBIES_CA_1        9
HOBBIES_CA_4        3
HOBBIES_WI_1        3
HOBBIES_TX_3        2
HOBBIES_TX_2        2
HOBBIES_WI_3        2
HOBBIES_CA_2        2
HOBBIES_TX_1        1
HOUSEHOLD_CA_4      1
Name: cat_store_id, dtype: int64


,item_id,cat_id,store_id,cat_store_id,d_1,d_2,d_3,d_4,d_5,d_6,...,s_1903,s_1904,s_1905,s_1906,s_1907,s_1908,s_1909,s_1910,s_1911,s_1912
104,FOODS_1_012,FOODS,TX_1,FOODS_TX_1,0,0,0,0,0,0,...,6.27,6.27,6.27,6.27,6.27,6.27,6.27,6.27,6.27,6.27
105,FOODS_1_012,FOODS,TX_2,FOODS_TX_2,0,0,0,0,0,0,...,6.27,6.27,6.27,6.27,6.27,6.27,6.27,6.27,6.27,6.27
106,FOODS_1_012,FOODS,TX_3,FOODS_TX_3,0,0,0,0,0,0,...,6.27,6.27,6.27,6.27,6.27,6.27,6.27,6.27,6.27,6.27
160,FOODS_1_018,FOODS,CA_1,FOODS_CA_1,17,3,8,2,5,0,...,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98
161,FOODS_1_018,FOODS,CA_2,FOODS_CA_2,4,1,5,4,3,3,...,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98,0.98
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26932,HOUSEHOLD_2_160,HOUSEHOLD,CA_3,HOUSEHOLD_CA_3,0,0,0,0,0,0,...,3.93,3.93,3.93,3.93,3.93,3.93,3.93,3.93,3.93,3.93
28822,HOUSEHOLD_2_350,HOUSEHOLD,CA_3,HOUSEHOLD_CA_3,5,5,1,5,7,0,...,2.94,2.94,2.94,2.94,2.94,2.94,2.94,2.94,2.94,2.94
29412,HOUSEHOLD_2_409,HOUSEHOLD,CA_3,HOUSEHOLD_CA_3,0,0,0,0,0,0,...,3.93,3.93,3.93,3.93,3.93,3.93,3.93,3.93,3.93,3.93
29592,HOUSEHOLD_2_427,HOUSEHOLD,CA_3,HOUSEHOLD_CA_3,2,4,0,2,0,2,...,3.48,3.48,3.48,3.48,3.48,3.48,3.48,3.48,3.48,3.48


# Seasonal-trend decomposition

In [13]:
def process_data(i):
    data = sales_items.values[i][4:4+1913]
    prices = sales_items.values[i][4+1913:]
    sd = STL(data, period=7).fit()
    seasonal = np.array(sd.seasonal).astype(np.float32)
    trend = np.array(sd.trend).astype(np.float32)
    sale = seasonal+trend
    sell_price = prices
    cat_id = sales_items.values[i][1]
    store_id = sales_items.values[i][2]
    cat_store_id = sales_items.values[i][3]
    return sale, sell_price, cat_id, store_id, cat_store_id


def parallel_process_data(num_processes):
    res = Parallel(n_jobs=num_processes)(delayed(process_data)(i) for i in tqdm(range(1150), total=1150))
    return res


num_processes = multiprocessing.cpu_count()
res = parallel_process_data(num_processes)
res = np.array(res)
sales, sell_prices, cat_id, store_id, cat_store_id = res[:,0], res[:,1], res[:,2], res[:,3], res[:,4]
sales = np.vstack(sales).astype(np.float32)
sell_prices = np.vstack(sell_prices).astype(np.float32)
cat_id = np.vstack(cat_id).squeeze(1)
store_id = np.vstack(store_id).squeeze(1)
cat_store_id = np.vstack(cat_store_id).squeeze(1)

del res
print(sales.shape, sell_prices.shape, cat_id.shape, store_id.shape, cat_store_id.shape)
np.save(os.path.join('datasets', 'm5-forecasting-accuracy', 'sales.npy'), sales)
np.save(os.path.join('datasets', 'm5-forecasting-accuracy', 'sell_prices.npy'), sell_prices)
np.save(os.path.join('datasets', 'm5-forecasting-accuracy', 'cat_id.npy'), cat_id)
np.save(os.path.join('datasets', 'm5-forecasting-accuracy', 'store_id.npy'), store_id)
np.save(os.path.join('datasets', 'm5-forecasting-accuracy', 'cat_store_id.npy'), cat_store_id)

100%|██████████| 1150/1150 [00:44<00:00, 25.78it/s]


(1150, 1913) (1150, 1913) (1150,) (1150,) (1150,)


/tmp/ipykernel_2805728/3362660936.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  res = np.array(res)


# Calendar

In [14]:
calendar = pd.read_csv('./datasets/m5-forecasting-accuracy/calendar.csv')
calendar = calendar[:1913]
calendar = calendar[['date', 'event_name_1', 'event_type_1']]
calendar['date'] = pd.to_datetime(calendar.date)
display(calendar)
print(len(calendar.event_name_1.unique()), calendar.event_name_1.unique())
print(len(calendar.event_type_1.unique()), calendar.event_type_1.unique())

,date,event_name_1,event_type_1
0,2011-01-29,NaN,NaN
1,2011-01-30,NaN,NaN
2,2011-01-31,NaN,NaN
3,2011-02-01,NaN,NaN
4,2011-02-02,NaN,NaN
...,...,...,...
1908,2016-04-20,NaN,NaN
1909,2016-04-21,NaN,NaN
1910,2016-04-22,NaN,NaN
1911,2016-04-23,NaN,NaN


31 [nan 'SuperBowl' 'ValentinesDay' 'PresidentsDay' 'LentStart' 'LentWeek2'
 'StPatricksDay' 'Purim End' 'OrthodoxEaster' 'Pesach End' 'Cinco De Mayo'
 "Mother's day" 'MemorialDay' 'NBAFinalsStart' 'NBAFinalsEnd'
 "Father's day" 'IndependenceDay' 'Ramadan starts' 'Eid al-Fitr'
 'LaborDay' 'ColumbusDay' 'Halloween' 'EidAlAdha' 'VeteransDay'
 'Thanksgiving' 'Christmas' 'Chanukah End' 'NewYear' 'OrthodoxChristmas'
 'MartinLutherKingDay' 'Easter']
5 [nan 'Sporting' 'Cultural' 'National' 'Religious']


In [15]:
calendar['day_of_week'] = calendar.date.apply(lambda x: x.dayofweek)
calendar['day_of_month'] = calendar.date.apply(lambda x: x.day) 
calendar['day_of_year'] = calendar.date.apply(lambda x: x.dayofyear)
calendar['month_of_year'] = calendar.date.apply(lambda x: x.month)
calendar['week_of_year'] = calendar.date.apply(lambda x: x.weekofyear)

calendar = calendar[['date', 'day_of_week', 'day_of_month', 'day_of_year', 'month_of_year', 'week_of_year', 'event_name_1', 'event_type_1']]

display(calendar)

,date,day_of_week,day_of_month,day_of_year,month_of_year,week_of_year,event_name_1,event_type_1
0,2011-01-29,5,29,29,1,4,NaN,NaN
1,2011-01-30,6,30,30,1,4,NaN,NaN
2,2011-01-31,0,31,31,1,5,NaN,NaN
3,2011-02-01,1,1,32,2,5,NaN,NaN
4,2011-02-02,2,2,33,2,5,NaN,NaN
...,...,...,...,...,...,...,...,...
1908,2016-04-20,2,20,111,4,16,NaN,NaN
1909,2016-04-21,3,21,112,4,16,NaN,NaN
1910,2016-04-22,4,22,113,4,16,NaN,NaN
1911,2016-04-23,5,23,114,4,16,NaN,NaN


In [16]:
dict_event_name_1 = dict.fromkeys(calendar.event_name_1.unique())
for i in range(len(dict_event_name_1)):
    key = list(dict_event_name_1.keys())[i]
    dict_event_name_1[key] = i

def encode_event_name_1(event_name):
    return dict_event_name_1[event_name]

dict_event_type_1 = dict.fromkeys(calendar.event_type_1.unique())
for i in range(len(dict_event_type_1)):
    key = list(dict_event_type_1.keys())[i]
    dict_event_type_1[key] = i

def encode_event_type_1(event_type):
    return dict_event_type_1[event_type]

In [17]:
calendar.event_name_1 = calendar.event_name_1.apply(encode_event_name_1) # transform event_name
calendar.event_type_1 = calendar.event_type_1.apply(encode_event_type_1) # transform event_type
display(calendar)
print(calendar.event_name_1.unique())
print(calendar.event_type_1.unique())

,date,day_of_week,day_of_month,day_of_year,month_of_year,week_of_year,event_name_1,event_type_1
0,2011-01-29,5,29,29,1,4,0,0
1,2011-01-30,6,30,30,1,4,0,0
2,2011-01-31,0,31,31,1,5,0,0
3,2011-02-01,1,1,32,2,5,0,0
4,2011-02-02,2,2,33,2,5,0,0
...,...,...,...,...,...,...,...,...
1908,2016-04-20,2,20,111,4,16,0,0
1909,2016-04-21,3,21,112,4,16,0,0
1910,2016-04-22,4,22,113,4,16,0,0
1911,2016-04-23,5,23,114,4,16,0,0


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30]
[0 1 2 3 4]


In [18]:
calendar['day_of_week'] = calendar.date.apply(lambda x: x.dayofweek / 6.0 - 0.5)
calendar['day_of_month'] = calendar.date.apply(lambda x: x.day / 30.0 - 0.5) 
calendar['day_of_year'] = calendar.date.apply(lambda x: x.dayofyear / 364.0 - 0.5)
calendar['month_of_year'] = calendar.date.apply(lambda x: x.month / 11.0 - 0.5)
calendar['week_of_year'] = calendar.date.apply(lambda x: x.weekofyear / 51.0 - 0.5)

calendar['event_name_1'] = calendar.event_name_1.apply(lambda x: x / calendar.event_name_1.max() - 0.5)
calendar['event_type_1'] = calendar.event_type_1.apply(lambda x: x / calendar.event_type_1.max() - 0.5)

display(calendar)

,date,day_of_week,day_of_month,day_of_year,month_of_year,week_of_year,event_name_1,event_type_1
0,2011-01-29,0.333333,0.466667,-0.420330,-0.409091,-0.421569,-0.5,-0.5
1,2011-01-30,0.500000,0.500000,-0.417582,-0.409091,-0.421569,-0.5,-0.5
2,2011-01-31,-0.500000,0.533333,-0.414835,-0.409091,-0.401961,-0.5,-0.5
3,2011-02-01,-0.333333,-0.466667,-0.412088,-0.318182,-0.401961,-0.5,-0.5
4,2011-02-02,-0.166667,-0.433333,-0.409341,-0.318182,-0.401961,-0.5,-0.5
...,...,...,...,...,...,...,...,...
1908,2016-04-20,-0.166667,0.166667,-0.195055,-0.136364,-0.186275,-0.5,-0.5
1909,2016-04-21,0.000000,0.200000,-0.192308,-0.136364,-0.186275,-0.5,-0.5
1910,2016-04-22,0.166667,0.233333,-0.189560,-0.136364,-0.186275,-0.5,-0.5
1911,2016-04-23,0.333333,0.266667,-0.186813,-0.136364,-0.186275,-0.5,-0.5


In [19]:
calendar = calendar.values[:, 1:].astype(np.float32)
print(calendar.shape)

(1913, 7)


In [20]:
np.save(os.path.join('datasets', 'm5-forecasting-accuracy', 'calendar.npy'), calendar)